In [1]:
import sys
sys.path.append('/home/daniel/Documents/Uni/MT/poi-prediction/')
import pandas as pd
from src.eval import create_prediction_df, compute_overall_metrics, create_prediction_poi_files

In [2]:
dm_path = '/home/daniel/Documents/Uni/MT/experiment_logs/joint/no-surface/gr_sagittal_im_all/freeze/SA-DenseNet-PatchTransformer/version_0/data_module_params.json'
model_path = '/home/daniel/Documents/Uni/MT/experiment_logs/joint/no-surface/gr_sagittal_im_all/freeze/SA-DenseNet-PatchTransformer/version_0/checkpoints/sad-pt-epoch=21-fine_mean_distance_val=27.01.ckpt'
val_df = create_prediction_df(dm_path, model_path, joint=True, split = 'val')
test_df = create_prediction_df(dm_path, model_path, joint=True, split = 'test')

In [3]:
#Drop the rows where loss_mask is false
val_df = val_df[val_df.loss_mask]
test_df = test_df[test_df.loss_mask]

In [5]:
poi_paths_val = create_prediction_poi_files(dm_path, model_path, poi_file_ending='test_poi.json', save_path='/home/daniel/prediction_files/', split='val', return_paths = True, project=False, joint = True)
poi_paths_test = create_prediction_poi_files(dm_path, model_path, poi_file_ending='test_poi.json', save_path='/home/daniel/prediction_files/', split='test', return_paths = True, project=False, joint = True)

In [6]:
#Drop all the rows where loss_mask is False
val_df_implants = val_df[val_df['poi_idx'].isin([90,91,94,95])]
test_df_implants = test_df[test_df['poi_idx'].isin([90,91,94,95])]

val_df_gruber = val_df[~val_df['poi_idx'].isin([90,91,94,95])]
test_df_gruber = test_df[~test_df['poi_idx'].isin([90,91,94,95])]

In [7]:
#Add a column split to the dataframes then concatenate them
val_metrics_implants = compute_overall_metrics(val_df_implants)
val_metrics_implants['Split'] = 'Val'
test_metrics_implants = compute_overall_metrics(test_df_implants)
test_metrics_implants['Split'] = 'Test'

metrics_implants = pd.concat([val_metrics_implants, test_metrics_implants])

val_metrics_gruber = compute_overall_metrics(val_df_gruber)
val_metrics_gruber['Split'] = 'Val'
test_metrics_gruber = compute_overall_metrics(test_df_gruber)
test_metrics_gruber['Split'] = 'Test'

metrics_gruber = pd.concat([val_metrics_gruber, test_metrics_gruber])
print(metrics_gruber)

                    Mean Error  Median Error        MSE  Accuracy  Max Error  \
coarse_error          3.031693      2.857999  11.444552  0.301136   7.859975   
refined_error         2.780748      2.576405  10.070773  0.386364   7.986739   
coarse_proj_error     2.752869      2.558195   9.539246  0.352273   9.292840   
refined_proj_error    2.637073      2.402610   9.030883  0.403409   9.292840   
coarse_error          3.447978      3.198934  14.589814  0.188776   9.030368   
refined_error         3.228405      2.919386  13.309847  0.275510   8.724882   
coarse_proj_error     3.054769      2.766286  12.095086  0.295918   9.473424   
refined_proj_error    3.057624      2.663897  12.172393  0.306122   8.226042   

                   Split  
coarse_error         Val  
refined_error        Val  
coarse_proj_error    Val  
refined_proj_error   Val  
coarse_error        Test  
refined_error       Test  
coarse_proj_error   Test  
refined_proj_error  Test  


In [6]:
#Make the index column a regular column called 'Prediction Type'
metrics_implants.reset_index(inplace=True)
metrics_implants.rename(columns={'index': 'Prediction Type'}, inplace=True)
#Rename the entries in the 'Prediction Type' column coarse_error -> Coarse, refined_error -> Fine, coarse_proj_error -> Coarse with projection, refined_proj_error -> Fine with projection
metrics_implants['Prediction Type'] = metrics_implants['Prediction Type'].replace({'coarse_error': 'Coarse', 'refined_error': 'Fine', 'coarse_proj_error': 'Coarse with projection', 'refined_proj_error': 'Fine with projection'})

# Reorder the columns for pretty printing
metrics = metrics_implants[['Split', 'Prediction Type', 'Mean Error', 'Median Error', 'MSE', 'Accuracy', 'Max Error']]

In [7]:
print(metrics.to_latex(index = False, float_format = '%.2f'))

\begin{tabular}{llrrrrr}
\toprule
Split & Prediction Type & Mean Error & Median Error & MSE & Accuracy & Max Error \\
\midrule
Val & Coarse & 5.88 & 5.47 & 46.11 & 0.05 & 13.50 \\
Val & Fine & 5.29 & 4.29 & 38.19 & 0.10 & 12.89 \\
Val & Coarse with projection & 6.62 & 5.98 & 53.30 & 0.00 & 13.22 \\
Val & Fine with projection & 5.96 & 5.36 & 43.89 & 0.00 & 12.41 \\
Test & Coarse & 6.70 & 6.36 & 56.39 & 0.00 & 15.80 \\
Test & Fine & 6.49 & 5.98 & 53.01 & 0.00 & 16.51 \\
Test & Coarse with projection & 6.41 & 6.32 & 47.53 & 0.05 & 12.20 \\
Test & Fine with projection & 6.65 & 6.96 & 50.32 & 0.05 & 12.20 \\
\bottomrule
\end{tabular}



In [16]:
from BIDS import POI
import numpy as np

def get_angles(poi_paths, k):
    sub, vert = k
    poi_pred = POI.load(poi_paths[sub, vert]['pred'])
    poi_gt = POI.load(poi_paths[sub, vert]['gt'])

    s94_90_pred = np.array(poi_pred[vert, 94]) - np.array(poi_pred[vert, 90])
    s94_90_gt = np.array(poi_gt[vert, 94]) - np.array(poi_gt[vert, 90])

    s_95_91_pred = np.array(poi_pred[vert, 95]) - np.array(poi_pred[vert, 91])
    s_95_91_gt = np.array(poi_gt[vert, 95]) - np.array(poi_gt[vert, 91])

    theta94_90 = np.arccos(np.dot(s94_90_pred, s94_90_gt) / (np.linalg.norm(s94_90_pred) * np.linalg.norm(s94_90_gt))) * 180 / np.pi
    theta95_91 = np.arccos(np.dot(s_95_91_pred, s_95_91_gt) / (np.linalg.norm(s_95_91_pred) * np.linalg.norm(s_95_91_gt))) * 180 / np.pi

    return (theta94_90, theta95_91)

all_angles_val = []
for k in poi_paths_val.keys():
    if k[0].startswith('W'):
        continue
    for i in range(3):
        all_angles_val.append(get_angles(poi_paths_val, k))

all_angles = np.array(all_angles_val)
print(all_angles.mean(), all_angles.std())

all_angles_test = []
for k in poi_paths_test.keys():
    #skip if k[0] starts with 'W'
    if k[0].startswith('W'):
        continue
    for i in range(3):
        all_angles_test.append(get_angles(poi_paths_test, k))

all_angles = np.array(all_angles_test)
print(all_angles.mean(), all_angles.std())

#Add a column split to the dataframes then concatenate them
val_metrics = compute_overall_metrics(val_df)
val_metrics['Split'] = 'Val'
test_metrics = compute_overall_metrics(test_df)
test_metrics['Split'] = 'Test'

metrics = pd.concat([val_metrics, test_metrics])
print(metrics.to_latex())

14.265836394505934 6.838617415880337
20.475023203122227 9.79160597181352
\begin{tabular}{lrrrrrl}
\toprule
 & Mean Error & Median Error & MSE & Accuracy & Max Error & Split \\
\midrule
coarse_error & 3.322457 & 2.949973 & 14.982346 & 0.275510 & 13.501968 & Val \\
refined_error & 3.036737 & 2.787781 & 12.939702 & 0.357143 & 12.893449 & Val \\
coarse_proj_error & 3.147389 & 2.816430 & 14.004633 & 0.316327 & 13.224525 & Val \\
refined_proj_error & 2.976233 & 2.784793 & 12.587449 & 0.362245 & 12.408307 & Val \\
coarse_error & 3.749224 & 3.349405 & 18.460030 & 0.171296 & 15.803191 & Test \\
refined_error & 3.530542 & 3.248445 & 16.985977 & 0.250000 & 16.509827 & Test \\
coarse_proj_error & 3.365700 & 2.898294 & 15.375907 & 0.273148 & 12.198874 & Test \\
refined_proj_error & 3.390417 & 2.858277 & 15.704583 & 0.282407 & 12.198874 & Test \\
\bottomrule
\end{tabular}

